In [151]:
import talib
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime
import MetaTrader5 as mt5
import pandas_ta as ta
from scipy.signal import savgol_filter, find_peaks
pd.set_option('mode.chained_assignment', None)
import time

In [152]:
if not mt5.initialize(login=114781990, server="Exness-MT5Trial6",password="Mypassword$1234"):
            print("initialize() failed, error code =",mt5.last_error())
            quit()
        
login = 114781990
password = 'Mypassword$1234'
server = 'Exness-MT5Trial6'

mt5.login(login, password, server)

rate = mt5.copy_rates_from('EURUSDm', mt5.TIMEFRAME_M15, datetime.datetime.now(), 3000)

nf = pd.DataFrame(rate)
#         print('getting data')

nf['date']=pd.to_datetime(nf['time'], unit='s')

nf.set_index(np.arange(len(nf)), inplace = True)
nf = nf.drop(['spread', 'real_volume', 'tick_volume', 'time'], axis='columns')


# nf.to_excel('EURUSD.xlsx')

df = nf.iloc[:500]  # First half
df2 = nf.iloc[500:]  # Second half


# print(df.atr.iloc[-1])

df['trend'] = 'no trend'



In [153]:
%matplotlib inline

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from IPython.display import HTML
pd.set_option('mode.chained_assignment', None)


def show(peaks_idx, troughs_idx):
    
    global df
    fig, ax = plt.subplots()
    plt.xticks(rotation=-30)

    price, = ax.plot(df.index, df.close, c='grey', lw=2, alpha=0.5, zorder=5)
    price_smooth, = ax.plot(df.index, df.close_smooth, c='b', lw=2, zorder=5)
    peaks, = ax.plot(df.index[peaks_idx], df.close_smooth.iloc[peaks_idx], \
                    c="r", linestyle="None", markersize=10.0, marker="o", zorder=10)

    troughs, = ax.plot(df.index[troughs_idx], df.close_smooth.iloc[troughs_idx], \
                    c="g", linestyle="None", markersize=10.0, marker="o", zorder=10)

    plt.show()

In [154]:
def find_trend():
    
    global df
    
#     print(df.head())

    df['atr'] = ta.atr(high=df.high, low=df.low, close=df.close)

#     df.dropna(inplace = True)
    
    
    if len(df) - 1 > 30:  # Ensure window_length is within bounds
        
    
        df['close_smooth'] = savgol_filter(df.close, 40, 10)

    #     df['close_smooth'] = savgol_filter(df.close, 30, 10)

        atr = df.atr.iloc[-1]

    #     print('atr: ', atr)

    #     List which contains all peaks index value
        peaks_idx,_ = find_peaks(df.close_smooth, distance = 40, width = 2, prominence= atr)

    #     List which contains all troughs index value
        troughs_idx,_ = find_peaks(-1*df.close_smooth, distance = 40, width = 2, prominence= atr)

        print('peaks_idx: ', len(peaks_idx))
        print('troughs_idx: ', len(troughs_idx))
        print('--------------------------')
        
        show(peaks_idx, troughs_idx)
        
#         time.sleep(10)

#         if len(peaks_idx) > 1 and len(troughs_idx) > 1:

#             current_high = df.close_smooth.iloc[peaks_idx[-1]]

#             prev_high = df.close_smooth.iloc[peaks_idx[-2]]

#             current_low = df.close_smooth.iloc[troughs_idx[-1]]

#             prev_low = df.close_smooth.iloc[troughs_idx[-2]]
            
#             print('C_high: ', current_high)
#             print('C_low: ', current_low)
#             print('Prev_H: ', prev_high)
#             print('Prev_L: ', prev_low)

#             if current_high > prev_high and current_low > prev_low:
                
#                 print('up')

#                 return 'up'

#             elif current_high < prev_high and current_low < prev_low:
                
#                 print('down')

#                 return 'down'

#     else:
        
#         print('no trend')

#         return 'no trend'


In [155]:
#  Caller
past_df = pd.DataFrame()
def Caller():
    
    global df, df2, past_df
    # Initialize an empty DataFrame to store the past data that gets dropped
    

#         Loop for taking the live data
    for index, row in df2.iterrows():
        # Add live data to the df DataFrame
        df = df.append(row, ignore_index=True)
#             self.df = pd.concat([self.df, row], ignore_index=True)

        # Delete the first row if the DataFrame has more than 202 rows
        if len(df) > 500:
            # Save the dropped row in past_df
            past_df = past_df.append(df.iloc[0], ignore_index=True)
            df = df.iloc[1:].reset_index(drop=True)
        
        print('DF: ', len(df))
#         df.loc[len(df)-1, 'trend'] =
        find_trend()
        

    result_df = pd.concat([past_df, df], ignore_index=True)

    result_df.to_excel('combined.xlsx')

In [ ]:
Caller()